# Learning Google Sheets API with Python

This notebook will guide you through the basics of interacting with Google Sheets using Python, `gspread`, and a Service Account.

## Table of Contents
1. [Prerequisites](#Prerequisites)
2. [Authentication](#Authentication)
3. [Open a Spreadsheet](#Open-a-Spreadsheet)
4. [Get Data](#Get-Data)
5. [Append Data](#Append-Data)

---

## Prerequisites

Before running this code, you need to:

1. **Create a Google Cloud Project** on the [Google Cloud Console](https://console.cloud.google.com/).
2. **Enable APIs**: Enable 'Google Sheets API' and 'Google Drive API'.
3. **Create a Service Account**:
   - Go to 'IAM & Admin' > 'Service Accounts'.
   - Create a service account and download the **JSON Key file**.
4. **Share your Google Sheet**: Open the Google Sheet you want to use and click 'Share'. Add the service account's email address (found in your JSON file) with 'Editor' permissions.
5. **Install Libraries**: Run `pip install gspread oauth2client python-dotenv`.

In [1]:
import os
import json
import gspread
from google.oauth2.service_account import Credentials
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

print("Libraries imported successfully!")

Libraries imported successfully!


## Authentication

In this example, we assume your Service Account JSON content is stored in an environment variable named `GOOGLE_SHEETS_CREDENTIALS`.

> **Note**: Storing the whole JSON in an ENV variable is common for Cloud Platforms like Heroku or Vercel. Alternatively, you can save the path to the file in `GOOGLE_APPLICATION_CREDENTIALS`.

In [2]:
def get_gspread_client():
    # Define the scope
    scope = [
        'https://www.googleapis.com/auth/spreadsheets',
        'https://www.googleapis.com/auth/drive'
    ]
    
    # Get credentials from environment variable
    creds_json = os.getenv('GOOGLE_SHEETS_CREDENTIALS')
    
    if not creds_json:
        raise ValueError("GOOGLE_SHEETS_CREDENTIALS environment variable NOT found.")
    
    # If the env var is a path to a file, load it directly
    if os.path.isfile(creds_json):
        creds = Credentials.from_service_account_file(creds_json, scopes=scope)
    else:
        # Otherwise, assume it's the JSON content itself
        info = json.loads(creds_json)
        creds = Credentials.from_service_account_info(info, scopes=scope)
        
    client = gspread.authorize(creds)
    return client

try:
    gc = get_gspread_client()
    print("Authenticated successfully!")
except Exception as e:
    print(f"Authentication failed: {e}")

Authenticated successfully!


## Open a Spreadsheet

You can open a spreadsheet by its **Title** or its **URL**.

In [3]:
# Replace with your actual sheet name or URL
SHEET_NAME = "BANK_TEST_SHEET"

try:
    spreadsheet = gc.open(SHEET_NAME)
    # Or: spreadsheet = gc.open_by_url("https://docs.google.com/spreadsheets/d/.../edit")
    
    # Select the first worksheet
    worksheet = spreadsheet.get_worksheet(0)
    print(f"Opened spreadsheet: {spreadsheet.title}")
except Exception as e:
    print(f"Error opening spreadsheet: {e}")

Opened spreadsheet: BANK_TEST_SHEET


## Get Data

There are multiple ways to fetch data:
- `get_all_records()`: Returns a list of dictionaries (keys are headers).
- `get_all_values()`: Returns a list of lists (rows).
- `col_values(index)`: Returns all values in a specific column.

In [5]:
# 1. Get all records as List of Dictionaries
data = worksheet.get_all_records()
print("Records:")
for row in data[:5]: # Print first 5 rows
    print(row)

# 2. Get all values as List of Lists
values = worksheet.get_all_values()
print(f"\nTotal Rows: {len(values)}")

Records:
{'No': 1, 'Name': 'Bank'}
{'No': 2, 'Name': 'Mata'}

Total Rows: 3


## Append Data

Adding a new row at the end of the sheet is very easy with `append_row()`.

In [12]:
new_row = ["2024-05-20", "John Doe", "Python Learning", "Successful"]

try:
    # Append the row to the worksheet
    worksheet.append_rows([new_row], value_input_option='USER_ENTERED')
    print("Row appended successfully!")
except Exception as e:
    print(f"Error appending row: {e}")

Row appended successfully!
